In [67]:
%load_ext autoreload
%autoreload 2
# add . to module name
import sys
sys.path.append('../src/')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [86]:
import pandas as pd
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import folium
from package import storage, key, strtime
from package.tracer.folium import add_tracer_list_to_folium_map

In [69]:
stops_df = storage.read_df('../data/cleaned/stops.csv')
stops_gdf = gpd.GeoDataFrame(stops_df, geometry=gpd.points_from_xy(stops_df.stop_lon, stops_df.stop_lat))

In [70]:
earliest_arrival_times = pd.read_csv('../data/raptor_results/arrival_times.csv')
# earliest_arrival_times = storage.read_df('/home/moritz/dev/uni/mcr-py/src/testdata/output/arrival_times.csv')
earliest_arrival_times.sort_values(by=['arrival_time'])
earliest_arrival_times["stop_id"] = earliest_arrival_times["stop_id"].astype(str)
earliest_arrival_times = earliest_arrival_times.set_index('stop_id')
earliest_arrival_times["is_reached"] = earliest_arrival_times["arrival_time"] != "--:--:--"
earliest_arrival_times

,arrival_time,is_reached
stop_id,,
178,15:36:00,True
6,15:26:54,True
148,15:46:18,True
257,15:33:00,True
920,15:42:32,True
...,...,...
171,15:45:28,True
21,15:35:07,True
58,15:51:32,True


In [71]:
earliest_arrival_times.loc["835", "arrival_time"]

'15:33:27'

In [72]:
tracer_map = storage.read_any_dict("../data/raptor_results/tracer_map.pkl")[
    "tracer_map"
]
current = tracer_map["835"]
current


In [73]:
from package.output import enrich_raptor_trace_results

tracer_map = enrich_raptor_trace_results(
    "../data/raptor_results/", "../data/cologne_gtfs.zip"
)
current = tracer_map["880"]


In [74]:
sample_stop = stops_gdf.sample(1)

m = folium.Map(location=[sample_stop.stop_lat, sample_stop.stop_lon], zoom_start=12)
for i, row in stops_gdf.iterrows():
    eat = earliest_arrival_times.loc[row.stop_id]
    is_reached = eat["is_reached"]
    color = "green" if is_reached else "red"
    folium.CircleMarker(
        location=[row.stop_lat, row.stop_lon],
        radius=1,
        color=color,
        popup=f"{row.stop_name} ({row.stop_id}) @ {eat['arrival_time']}",
    ).add_to(m)
m
add_tracer_list_to_folium_map(current, m, stops_df)
m


/home/moritz/miniconda3/envs/mcr-py/lib/python3.10/site-packages/folium/utilities.py:69: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  float(coord)
/home/moritz/miniconda3/envs/mcr-py/lib/python3.10/site-packages/folium/utilities.py:77: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  if math.isnan(float(coord)):
/home/moritz/miniconda3/envs/mcr-py/lib/python3.10/site-packages/folium/utilities.py:79: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return [float(x) for x in coords]


## McRAPTOR results

In [102]:
labels_per_stop_id = storage.read_any_dict("../data/mc_raptor_results/mc_raptor_labels.pkl")[key.MC_RAPTOR_LABELS_KEY]

In [103]:
latest_arrival_time = strtime.str_time_to_seconds("16:30:00")
max_walking_time = strtime.str_time_to_seconds("00:10:00")

In [104]:
labels_per_stop_id = {
    stop_id: set(
        (arrival_time, walking_time)
        for (arrival_time, walking_time) in labels
        if strtime.str_time_to_seconds(arrival_time) <= latest_arrival_time
        and strtime.str_time_to_seconds(walking_time) <= max_walking_time
    )
    for stop_id, labels in labels_per_stop_id.items()
}


In [105]:
sample_stop = stops_gdf.sample(1)

m = folium.Map(location=[sample_stop.stop_lat, sample_stop.stop_lon], zoom_start=12)
for i, row in stops_gdf.iterrows():
    labels = labels_per_stop_id[row.stop_id]
    color = "green" if len(labels) else "red"
    folium.CircleMarker(
        location=[row.stop_lat, row.stop_lon],
        radius=1,
        color=color,
        popup=f"{row.stop_name} ({row.stop_id}) \n {labels}",
    ).add_to(m)
m

/home/moritz/miniconda3/envs/mcr-py/lib/python3.10/site-packages/folium/utilities.py:69: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  float(coord)
/home/moritz/miniconda3/envs/mcr-py/lib/python3.10/site-packages/folium/utilities.py:77: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  if math.isnan(float(coord)):
/home/moritz/miniconda3/envs/mcr-py/lib/python3.10/site-packages/folium/utilities.py:79: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return [float(x) for x in coords]
